<a href="https://colab.research.google.com/github/nguyenvantin971996/CreateData/blob/main/Backpropagation/8_nodes_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')
df=pd.read_csv('gdrive/My Drive/Colab Notebooks/data_8_nodes_yen.csv')
df.head()

Mounted at /content/gdrive


,"X(1, 2)","X(1, 3)","X(2, 4)","X(2, 5)","X(3, 4)","X(3, 7)","X(4, 6)","X(5, 6)","X(5, 8)","X(6, 7)",...,"Y4(1, 3)","Y4(2, 4)","Y4(2, 5)","Y4(3, 4)","Y4(3, 7)","Y4(4, 6)","Y4(5, 6)","Y4(5, 8)","Y4(6, 7)","Y4(7, 8)"
0,10,40,30,50,30,20,20,10,90,70,...,0,1,0,0,0,1,0,0,1,1
1,90,50,50,30,30,80,80,30,30,80,...,1,0,0,1,0,1,1,1,0,0
2,10,60,80,50,40,50,30,90,80,30,...,1,0,0,1,0,1,0,0,1,1
3,80,20,40,60,50,80,90,60,80,80,...,1,0,0,1,0,1,1,1,0,0
4,20,70,50,40,20,70,80,80,10,10,...,0,1,0,0,0,1,1,1,0,0


In [3]:
edges = [[1, 2], [1, 3], [2, 4], [2, 5], [3, 4], [3, 7], [4, 6], [5, 6], [5, 8], [6, 7], [7, 8]]
number_edges = len(edges)
number_steps = 4
start_node = 1
end_node = 8

In [4]:
X = df.iloc[:,:number_edges].values
y = df.iloc[:,number_edges:].values

In [5]:
print(X.shape, y.shape)

(100000, 11) (100000, 44)


In [6]:
from sklearn.model_selection import train_test_split
X1,X2,y_train,y_test = train_test_split(X,y,test_size = 0.1)

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X1)
X_test= sc.fit_transform(X2)

In [8]:
X_train = np.repeat(X_train[:, np.newaxis, : ], number_steps, axis=1)
X_test = np.repeat(X_test[:, np.newaxis, : ], number_steps, axis=1)
y_train_2 = np.reshape(y_train,(y_train.shape[0], number_steps, number_edges))
y_test_2 = np.reshape(y_test,(y_test.shape[0], number_steps, number_edges))

In [9]:
print(X_train.shape, y_train_2.shape)

(90000, 4, 11) (90000, 4, 11)


In [11]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,TimeDistributed,LSTM
model = Sequential()
model.add(LSTM(32, input_shape=(number_steps, number_edges), return_sequences=True))
model.add(LSTM(32, return_sequences=True))
model.add(TimeDistributed(Dense(number_edges, activation='sigmoid')))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 4, 32)             5632      
                                                                 
 lstm_3 (LSTM)               (None, 4, 32)             8320      
                                                                 
 time_distributed_1 (TimeDis  (None, 4, 11)            363       
 tributed)                                                       
                                                                 
Total params: 14,315
Trainable params: 14,315
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(loss='BinaryCrossentropy', optimizer='adam')
model.fit(X_train, y_train_2, epochs=100, batch_size=64)

Epoch 1/100
1407/1407 [==============================] - 16s 6ms/step - loss: 0.4250
Epoch 2/100
1407/1407 [==============================] - 8s 5ms/step - loss: 0.2184
Epoch 3/100
1407/1407 [==============================] - 7s 5ms/step - loss: 0.1647
Epoch 4/100
1407/1407 [==============================] - 8s 6ms/step - loss: 0.1375
Epoch 5/100
1407/1407 [==============================] - 8s 6ms/step - loss: 0.1204
Epoch 6/100
1407/1407 [==============================] - 8s 5ms/step - loss: 0.1083
Epoch 7/100
1407/1407 [==============================] - 14s 10ms/step - loss: 0.0992
Epoch 8/100
1407/1407 [==============================] - 7s 5ms/step - loss: 0.0922
Epoch 9/100
1407/1407 [==============================] - 10s 7ms/step - loss: 0.0861
Epoch 10/100
1407/1407 [==============================] - 10s 7ms/step - loss: 0.0812
Epoch 11/100
1407/1407 [==============================] - 7s 5ms/step - loss: 0.0770
Epoch 12/100
1407/1407 [==============================] - 7s 5ms/step

In [13]:
pred=model.predict(X_train)
pred_test=model.predict(X_test)

313/313 [==============================] - 1s 2ms/step


In [14]:
import copy
def cost_path(x,y):
    cost = 0
    path = []
    for i in range(number_edges):
        if y[i]==1:
            path.append(edges[i])
            cost+= x[i]
    if(len(path)!=0):
        path_2 = copy.deepcopy(path)
        path_nodes = [start_node]
        if(path[0][0]!=start_node):
            cost = 0
        else:
            path_nodes.append(path[0][1])
            del path_2[0]
            condition = True
            while(len(path_2)!=0 and condition):
                condition = False
                for i in range(len(path_2)):
                    if(path_nodes[-1] in path_2[i]):
                        path_2[i].remove(path_nodes[-1])
                        path_nodes.append(path_2[i][0])
                        path_2.pop(i)
                        condition = True
                        break
        if(path_nodes[-1]!=end_node or len(path_2)!=0):
            cost = 0
    else:
        cost = 0
    return cost

In [15]:
accuracy = np.zeros(number_steps+1)
for i in range(pred.shape[0]):
    condition_1 = True
    for t in range(number_steps):
        output = copy.deepcopy(pred[i][t])
        condition_2 = True
        for j in range(len(output)):
            output[j] = round(output[j])
        if(cost_path(X1[i],output) != cost_path(X1[i],y_train_2[i][t])):
            condition_2 = False
            condition_1 = False
        if condition_2:
            accuracy[t] += 1
    if condition_1:
        accuracy[number_steps] += 1
accuracy = np.round(accuracy / pred.shape[0] * 100,2)
accuracy

array([98.31, 97.33, 97.  , 94.08, 89.56])

In [16]:
accuracy_test = np.zeros(number_steps+1)
for i in range(pred_test.shape[0]):
    condition_1 = True
    for t in range(number_steps):
        output = copy.deepcopy(pred_test[i][t])
        condition_2 = True
        for j in range(len(output)):
            output[j] = round(output[j])
        if(cost_path(X1[i],output) != cost_path(X1[i],y_test_2[i][t])):
            condition_2 = False
            condition_1 = False
        if condition_2:
            accuracy_test[t] += 1
    if condition_1:
        accuracy_test[number_steps] += 1
accuracy_test = np.round(accuracy_test / pred_test.shape[0] * 100,2)
accuracy_test

array([97.9 , 96.51, 95.53, 91.46, 86.5 ])

In [17]:
model.summary()
print(accuracy)
print(accuracy_test)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 4, 32)             5632      
                                                                 
 lstm_3 (LSTM)               (None, 4, 32)             8320      
                                                                 
 time_distributed_1 (TimeDis  (None, 4, 11)            363       
 tributed)                                                       
                                                                 
Total params: 14,315
Trainable params: 14,315
Non-trainable params: 0
_________________________________________________________________
[98.31 97.33 97.   94.08 89.56]
[97.9  96.51 95.53 91.46 86.5 ]
